## Table of Contents
- [Gradient Boost Regressor](#Gradient-Boost-Regressor)
  - [Concept](#Concept)
  - [How It Works](#How-It-Works)
  - [Key Parameters](#Key-Parameters)
  - [Advantages](#Advantages)
  - [Applications](#Applications)
- [Implementation](#Implementation)
- [🏠 Home](../../../../welcomePage.ipynb)

# Gradient Boost Regressor

**Gradient Boosting Regressor** is a powerful ensemble machine learning algorithm that builds models in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.

## Concept

Gradient Boosting involves three main components:
1. **Loss Function to be optimized:** Gradient Boosting is a flexible method that can be used on differentiable loss functions. For regression tasks, it typically uses squared error or absolute error.
2. **Weak Learner to make predictions:** Gradient Boosting uses decision trees as the weak learners. Trees are added one at a time, and existing trees in the model are not changed.
3. **Additive Model to add weak learners:** Trees are added one at a time, and gradient descent is used to minimize the loss when adding trees.

## How It Works

The algorithm builds the model in a stage-wise fashion:
1. Fit a decision tree to the data, e.g., predict the mean of the target variable.
2. Apply the decision tree to the data and calculate the error residuals.
3. Fit a new decision tree to the residuals from the previous step.
4. Add this new decision tree into the ensemble, update the model.
5. Repeat steps 2-4 until a specified number of trees have been added or the loss changes minimally on adding a new tree.

## Key Parameters

- $n_{\text{estimators}}$: The number of boosting stages to perform. More stages increase the model complexity.
- $\text{learning\_rate}$: Shrinks the contribution of each tree by the learning rate. There is a trade-off between learning rate and number of stages.
- $\text{max\_depth}$: Limits the number of nodes in the decision trees. Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.
- $\text{min\_samples\_split}$: The minimum number of samples required to split an internal node.
- $\text{min\_samples\_leaf}$: The minimum number of samples required to be at a leaf node.

## Advantages

- Can handle heterogeneous features (numeric and categorical).
- Robust to outliers in output space (via robust loss functions).
- Provides predictive score distributions by way of quantile regression.

## Applications

Gradient Boosting can be used for:
- Demand forecasting in retail,
- Price prediction in real estate,
- Predicting customer lifetime value in various industries.


# Implementation

### Import Libraries

**Press ▶ to import libraries.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from IPython.display import display, clear_output, HTML

import warnings
warnings.filterwarnings("ignore")

print("Libraries are imported.")

### Import and show Data

**Press ▶ to load data.**

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# List all .csv and Excel files in the current directory
supported_extensions = ['.csv', '.xlsx', '.xls']
files = [f for f in os.listdir('./Data') if any(f.endswith(ext) for ext in supported_extensions)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=files,
    description='Files:',
    disabled=False,
)

# Create a button widget
button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select file',
    icon='check'
)

# Output widget to display messages
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        clear_output()
        selected_file = dropdown.value
        global data
        if selected_file.endswith('.csv'):
            data = pd.read_csv("./Data/" +selected_file)
        elif selected_file.endswith(('.xlsx', '.xls')):
            data = pd.read_excel("./Data/" +selected_file)
        print(f"File '{selected_file}' uploaded as data.")

# Attach the function to the button widget
button.on_click(on_button_click)

# Display the dropdown, button widgets, and initial message within the output widget
with output:
    print("Please select a file from the dropdown and click 'Select'.")
display(output)
display(dropdown)
display(button)



**Press ▶ to display the data.**

In [ ]:
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

### Data Preprocessing

**Press ▶ to specify the target column.**

In [ ]:
import ipywidgets as widgets
import pandas as pd

# Create a Dropdown widget for column selection
dropdown = widgets.Dropdown(
    options=data.columns.tolist(),
    value=data.columns[0],
    description='Select Target Column:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a Button widget
button = widgets.Button(
    description='Select',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to select the target column as the last column',
    icon='check'  # FontAwesome icon names (without 'fa-')
)

# Create an Output widget for displaying messages
output = widgets.Output()

# Function to handle button click that rearranges the DataFrame
def on_button_clicked(b):
    with output:
        output.clear_output()
        global data
        # Get the selected column name
        selected_column = dropdown.value
        # Reorder the DataFrame columns
        new_columns = [col for col in data.columns if col != selected_column] + [selected_column]
        data = data[new_columns]
        print(f"Column '{selected_column}' has been moved to the last position.")

# Link the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets and output
display(widgets.VBox([dropdown, button, output]))


**Press ▶ to create a lagged target column.**

In [ ]:
target = data.columns[-1]
data['Target_Lag'] = data.iloc[:, -1].shift(1)

data.dropna(inplace=True)

### Predict Bead Area

#### Parameters

##### Number of Estimators
The number of estimators refers to the number of boosting stages to be run, which is equivalent to the number of trees in the model. This parameter controls how many weak learners (typically decision trees) are added to the model sequentially. Each new tree is trained to correct the errors made by the combined ensemble of all previous trees. A higher number of estimators generally increases the complexity of the model and can lead to better performance, up to a point. However, too many estimators can lead to overfitting. By adding more estimators, the model can learn more intricate patterns in the data, improving prediction accuracy.

##### Maximum Depth
The maximum depth of the individual trees sets the maximum number of levels in each decision tree. This parameter limits how deep the trees can grow. Deeper trees can capture more detailed interactions in the data but can also lead to overfitting if they become too complex. Limiting the maximum depth helps prevent overfitting by controlling the complexity of the trees, while a suitable maximum depth allows the model to capture important interactions between features without becoming too complex.

##### Learning Rate
The learning rate is the step size at which the model learns, shrinking the contribution of each tree by a factor of the learning rate. This parameter scales the contribution of each new tree added to the ensemble. A smaller learning rate requires more trees to model the data effectively but can lead to better generalization. A lower learning rate generally improves the model's performance by making it learn more slowly and steadily, thus preventing overfitting. However, it requires more estimators to achieve the same level of performance. Balancing the learning rate with the number of estimators is crucial for building a robust model that generalizes well.

**Press ▶ to specify independent variables, train/test split, and the model parameter and to forecast the data.**

In [ ]:

# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, min(7000, len(data))],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

feature_select = widgets.SelectMultiple(
    options=tuple(col for col in data.columns if col != target),
    value=tuple(col for col in data.columns if col != target),
    description='Features:',
    layout=widgets.Layout(width='600px', height='180px'),  # Increase width and height
    style={'description_width': '150px'},  # Increase description width
    disabled=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

# Gradient Boosting parameter sliders
n_estimators_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=10,
    description='Number of Estimators:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '160px'},
    continuous_update=False
)

max_depth_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=20,
    step=1,
    description='Maximum Depth:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

learning_rate_slider = widgets.FloatSlider(
    value=0.1,
    min=0.01,
    max=1,
    step=0.01,
    description='Learning Rate:',
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'},
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        selected_features = list(feature_select.value)
        train_size_pct = train_size_slider.value / 100
        n_estimators = n_estimators_slider.value
        max_depth = max_depth_slider.value
        learning_rate = learning_rate_slider.value
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        

        X = df[selected_features]
        y = df[target]
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Train the model
        model = GradientBoostingRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel(target)
        plt.title('Actual vs Predicted '+target)
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test - y_pred) ** 2
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual '+target)
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, feature_select, train_size_slider, n_estimators_slider, max_depth_slider, learning_rate_slider, apply_button, output)


### <center>[🏠 Home](../../../../welcomePage.ipynb)</center>